# Rule_inconsistency_check

### Install rdflib library

In [4]:
#!pip install rdflib

### Import libraries

In [5]:
from rdflib import *
from rdflib import URIRef, Graph, Namespace

### Import T4 project knowledge base

In [6]:
filepath = '/content/drive/MyDrive/Colab Notebooks/SPB_PU_Amir/data/Project_knowledge_base_v1.n3'

In [7]:
f = open(filepath, "r")
print(f.read())

@prefix ind:<URN:inds:>.
@prefix prop:<URN:prop:>.
@prefix classes:<URN:class>.
@prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>.
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .


ind:process0 rdf:type classes:Process;
		rdfs:label "Detecting lung diseases from medical imaging";
		rdf:isDefinedBy	"T4 project group";
		prop:hasInput ind:kaggledataset;
        prop:hasInput ind:messagewithimage;
        prop:hasOutput ind:text;
        prop:hasKPI	ind:kpi0;
        prop:hasKPI	ind:kpi1;
        prop:hasKPI	ind:kpi2;
        prop:hasKPI	ind:kpi3;
        prop:hasResource ind:resource0;
        prop:hasResource ind:resource1;
        prop:hasResource ind:resource2.

ind:process01 prop:SubProcess ind:process0;
		rdfs:label "Dataset and input Vectorizing";
        prop:hasInput ind:kaggledataset;
        prop:hasInput ind:messagewithimage;
        prop:hasResource ind:resource1;
        prop:hasResource ind:resource1.

ind:proc

#### Parsing the knowledge base

In [8]:
knowledge_base = Graph()

In [9]:
results = knowledge_base.parse(filepath, format="text/n3")
results

<Graph identifier=N48b69129e18e456285eb293f37966b85 (<class 'rdflib.graph.Graph'>)>

In [10]:
print(len(results))

60


In [11]:
print(results.serialize(format="text/n3"))

@prefix classes: <URN:class> .
@prefix ind: <URN:inds:> .
@prefix prop: <URN:prop:> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ind:process01 rdfs:label "Dataset and input Vectorizing" ;
    prop:SubProcess ind:process0 ;
    prop:hasInput ind:kaggledataset,
        ind:messagewithimage ;
    prop:hasResource ind:resource1 .

ind:process02 rdfs:label "Neural network construction" ;
    prop:SubProcess ind:process0 ;
    prop:hasInput ind:process01 ;
    prop:hasKPI ind:kpi0,
        ind:kpi1,
        ind:kpi2 ;
    prop:hasOutput ind:process03 ;
    prop:hasResource ind:resource2 .

ind:process04 rdfs:label "Result output" ;
    prop:SubProcess ind:process0 ;
    prop:hasInput ind:process03 ;
    prop:hasOutput ind:text ;
    prop:hasResource ind:resource0,
        ind:resource1 .

ind:kaggledataset a classes:inputs ;
    rdfs:label "kaggledataset" .

ind:kpi0 a classes:KPI ;
    rdfs:label "Design topology" .


### Import T4 project rule base

In [12]:
filepath = '/content/drive/MyDrive/Colab Notebooks/SPB_PU_Amir/data/Rules_v2.n3'

In [13]:
f = open(filepath, "r")
print(f.read())

@prefix ind:<URN:inds:>.
@prefix prop:<URN:prop:>.
@prefix classes:<URN:classes:>.
@prefix rdfs:<http://www.w3.org/2000/01/rdf-schema#>.
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .


ind:KPI	a owl:Class;
		owl:oneof ind:kpi0,ind:kpi1,ind:kpi2,ind:kpi3.

ind:resources a owl:Class;
		owl:oneof ind:resource0,ind:resource1,ind:resource2.


ind:KPI a owl:Class;
owl:disjointWith ind:resources.

ind:resources a owl:Class;
owl:disjointWith ind:KPI.


ind:hasKPI a owl:ObjectProperty;
a owl:IrreflexiveProperty.

ind:hasResource a owl:ObjectProperty;
a owl:AsymmetricProperty.


#### Parsing the rule base

In [14]:
rule_base = Graph()

In [15]:
results = rule_base.parse(filepath, format="text/n3")
results

<Graph identifier=Nce5a79ebd8324a1a815acbe60e649ed8 (<class 'rdflib.graph.Graph'>)>

In [16]:
print(len(results))

15


In [17]:
print(results.serialize(format="text/n3"))

@prefix ind: <URN:inds:> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

ind:hasKPI a owl:IrreflexiveProperty,
        owl:ObjectProperty .

ind:hasResource a owl:AsymmetricProperty,
        owl:ObjectProperty .

ind:KPI a owl:Class ;
    owl:disjointWith ind:resources ;
    owl:oneof ind:kpi0,
        ind:kpi1,
        ind:kpi2,
        ind:kpi3 .

ind:resources a owl:Class ;
    owl:disjointWith ind:KPI ;
    owl:oneof ind:resource0,
        ind:resource1,
        ind:resource2 .




### Check the knowledge base for inconsistency

#### **owl**:oneOf

- Any individual that belongs to the class **KPI** must be either **'kpi0', 'kpi1', 'kpi2', 'kpi3'**, and no other individual can belong to the class **KPI**.
- Any individual that belongs to the class **resources** must be either **'resource0', 'resource1', 'resource2'**, and no other individual can belong to the class **'resources'**.

##### Query to rule base

In [18]:
rule_base_query_1 = rule_base.query(
            """SELECT DISTINCT ?t4class ?t4values
               WHERE {
                  ?t4class owl:oneof ?t4values.
               }""")

In [19]:
my_dict_1 = {'KPI':[],
           'resources':[]}
for row in rule_base_query_1:
  t4class = str(row.asdict()['t4class'].toPython().split(":")[2])
  t4values = str(row.asdict()['t4values'].toPython().split(":")[2])
  if t4class not in my_dict_1:
    my_dict_1[t4class] = t4values
  else:
    my_dict_1[t4class].append(t4values)

In [20]:
my_dict_1

{'KPI': ['kpi0', 'kpi1', 'kpi2', 'kpi3'],
 'resources': ['resource0', 'resource1', 'resource2']}

##### Query to knowledge base

In [21]:
knowledge_base_query_1 = knowledge_base.query(
"""
SELECT ?subject ?predicate ?object
WHERE {
  ?subject ?predicate ?object.
  FILTER (
    ?predicate = prop:hasKPI || ?predicate = prop:hasResource
  )
}
""")

In [22]:
my_dict_2 = {'hasKPI':[],
             'hasResource':[]}

for row in knowledge_base_query_1:
    predicate = str(row.asdict()['predicate'].toPython().split(":")[2])
    t4_object = str(row.asdict()['object'].toPython().split(":")[2])
    if predicate not in my_dict_2:
      my_dict_2[predicate] = t4_object
    else:
      my_dict_2[predicate].append(t4_object)

In [50]:
print(my_dict_2)

{'hasKPI': ['kpi1', 'kpi3', 'kpi2', 'kpi0', 'kpi2', 'kpi1', 'kpi0', 'kpi3', 'kpi2'], 'hasResource': ['resource0', 'resource1', 'resource1', 'resource2', 'resource2', 'resource0', 'resource0', 'resource1', 'resource2']}


##### Check for inconsistency

In [24]:
for key, value in my_dict_2.items():
  if key =='hasKPI':
    for item in value:
      if item in my_dict_1['KPI']:
        print(item+' --  in list')
      else:
        print("Inconsistency Found in item  "+item)
  if key =='hasResource':
    for item in value:
      if item in my_dict_1['resources']:
        print(item+' --  in list')
      else:
        print("Inconsistency Found in item  "+item)

kpi1 --  in list
kpi3 --  in list
kpi2 --  in list
kpi0 --  in list
kpi2 --  in list
kpi1 --  in list
kpi0 --  in list
kpi3 --  in list
kpi2 --  in list
resource0 --  in list
resource1 --  in list
resource1 --  in list
resource2 --  in list
resource2 --  in list
resource0 --  in list
resource0 --  in list
resource1 --  in list
resource2 --  in list


#### owl:disjointWith

Any individual that belongs to the class **KPI** cannot belong to the class **resources** and vice versa.

##### Query to rule base

In [25]:
rule_base_query_2 = rule_base.query(
            """SELECT DISTINCT ?t4class ?t4values
               WHERE {
                  ?t4class owl:disjointWith ?t4values.
               }""")

In [26]:
my_dict_3 = {}
for row in rule_base_query_2:
  t4class = str(row.asdict()['t4class'].toPython().split(":")[2])
  t4values = str(row.asdict()['t4values'].toPython().split(":")[2])
  my_dict_3[t4class] = t4values

In [27]:
my_dict_3

{'KPI': 'resources', 'resources': 'KPI'}

##### Query to knowledge base

In [28]:
knowledge_base_query_2 = knowledge_base.query(
"""
SELECT ?subject ?predicate ?object
WHERE {
  ?subject rdf:type ?object.
  FILTER (
    ?object = classes:KPI || ?object = classes:resources
  )
}
""")

In [29]:
my_dict_4 = {'classKPI':[],
              'classresources':[]}
for row in knowledge_base_query_2:
  t4_subject = str(row.asdict()['subject'].toPython().split(":")[2])
  t4_object = str(row.asdict()['object'].toPython().split(":")[1])
  if t4_object not in my_dict_4:
      my_dict_4[t4_object] = t4_subject
  else:
      my_dict_4[t4_object].append(t4_subject)

In [30]:
 my_dict_4

{'classKPI': ['kpi0', 'kpi1', 'kpi2', 'kpi3'],
 'classresources': ['resource0', 'resource1', 'resource2']}

##### Check for inconsistency

In [31]:
for key, value in my_dict_4.items():
  if key =='classKPI':
    for item in value:
      if item in my_dict_4['classresources']:
        print("Inconsistency Found in item  "+item)
      else:
        print(item, "found only in class", key)
  if key =='classresources':
    for item in value:
      if item in my_dict_4['classKPI']:
        print("Inconsistency Found in item  "+item)
      else:
        print(item, "found only in class", key)

kpi0 found only in class classKPI
kpi1 found only in class classKPI
kpi2 found only in class classKPI
kpi3 found only in class classKPI
resource0 found only in class classresources
resource1 found only in class classresources
resource2 found only in class classresources


#### owl:IrreflexiveProperty

Any individual that has the property **hasKPI** with another individual cannot have the same property with itself. Any statement using the property **hasKPI** cannot have the same resource as both the subject and the object.

##### Query to rule base

In [32]:
rule_base_query_3 = rule_base.query(
            """SELECT DISTINCT ?t4_subject
               WHERE {
                  ?t4_subject a owl:IrreflexiveProperty.
               }""")

In [33]:
for row in rule_base_query_3:
  print(row)

(rdflib.term.URIRef('URN:inds:hasKPI'),)


In [34]:
my_list_1 = []
for row in rule_base_query_3:
  t4_subject = str(row.asdict()['t4_subject'].toPython().split(":")[2])
  my_list_1.append(t4_subject)

In [35]:
my_list_1

['hasKPI']

##### Query to knowledge base

In [36]:
knowledge_base_query_3 = knowledge_base.query(
"""
SELECT ?subject ?object
WHERE {
  ?subject prop:hasKPI ?object.
  }
""")

In [37]:
for row in knowledge_base_query_3:
  print(row)

(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:kpi0'))
(rdflib.term.URIRef('URN:inds:process02'), rdflib.term.URIRef('URN:inds:kpi0'))
(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:kpi1'))
(rdflib.term.URIRef('URN:inds:process02'), rdflib.term.URIRef('URN:inds:kpi1'))
(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:kpi2'))
(rdflib.term.URIRef('URN:inds:process02'), rdflib.term.URIRef('URN:inds:kpi2'))
(rdflib.term.URIRef('URN:inds:process03'), rdflib.term.URIRef('URN:inds:kpi2'))
(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:kpi3'))
(rdflib.term.URIRef('URN:inds:process03'), rdflib.term.URIRef('URN:inds:kpi3'))


In [38]:
subject_list_1 =[]
object_list_1 = []
for row in knowledge_base_query_3:
  t4_subject = str(row.asdict()['subject'].toPython().split(":")[2])
  t4_object = str(row.asdict()['object'].toPython().split(":")[2])
  subject_list_1.append(t4_subject)
  object_list_1.append(t4_object)

In [39]:
object_list_1

['kpi0', 'kpi0', 'kpi1', 'kpi1', 'kpi2', 'kpi2', 'kpi2', 'kpi3', 'kpi3']

##### Check for inconsistency

In [51]:
for i in range(len(subject_list_1)):
  if subject_list_1[i] == object_list_1[i]:
    print("Inconsistency Found in pair  ",subject_list_1[i],object_list_1[i])
  else:
    print(subject_list_1[i],' and ', object_list_1[i] ,"are different")

process0  and  kpi0 are different
process02  and  kpi0 are different
process0  and  kpi1 are different
process02  and  kpi1 are different
process0  and  kpi2 are different
process02  and  kpi2 are different
process03  and  kpi2 are different
process0  and  kpi3 are different
process03  and  kpi3 are different


#### owl:AsymmetricProperty

 If one individual has **hasResource** property with another individual, then the second individual cannot have the same **hasResource** property with the first individual.

##### Query to rule base

In [41]:
rule_base_query_4 = rule_base.query(
            """SELECT DISTINCT ?t4_subject
               WHERE {
                  ?t4_subject a owl:AsymmetricProperty.
               }""")

In [42]:
for row in rule_base_query_4:
  print(row)

(rdflib.term.URIRef('URN:inds:hasResource'),)


In [43]:
my_list_2 = []
for row in rule_base_query_4:
  t4_subject = str(row.asdict()['t4_subject'].toPython().split(":")[2])
  my_list_2.append(t4_subject)

In [44]:
my_list_2

['hasResource']

##### Query to knowledge base

In [45]:
knowledge_base_query_4 = knowledge_base.query(
"""
SELECT ?subject ?object
WHERE {
  ?subject prop:hasResource ?object.
  }
""")

In [46]:
for row in knowledge_base_query_4:
  print(row)

(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:resource0'))
(rdflib.term.URIRef('URN:inds:process03'), rdflib.term.URIRef('URN:inds:resource0'))
(rdflib.term.URIRef('URN:inds:process04'), rdflib.term.URIRef('URN:inds:resource0'))
(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:resource1'))
(rdflib.term.URIRef('URN:inds:process01'), rdflib.term.URIRef('URN:inds:resource1'))
(rdflib.term.URIRef('URN:inds:process04'), rdflib.term.URIRef('URN:inds:resource1'))
(rdflib.term.URIRef('URN:inds:process0'), rdflib.term.URIRef('URN:inds:resource2'))
(rdflib.term.URIRef('URN:inds:process02'), rdflib.term.URIRef('URN:inds:resource2'))
(rdflib.term.URIRef('URN:inds:process03'), rdflib.term.URIRef('URN:inds:resource2'))


In [47]:
my_list_5 = []
for row in knowledge_base_query_4:
  t4_subject = str(row.asdict()['subject'].toPython().split(":")[2])
  t4_object = str(row.asdict()['object'].toPython().split(":")[2])
  my_list_5.append((t4_subject,t4_object))

In [48]:
my_list_5

[('process0', 'resource0'),
 ('process03', 'resource0'),
 ('process04', 'resource0'),
 ('process0', 'resource1'),
 ('process01', 'resource1'),
 ('process04', 'resource1'),
 ('process0', 'resource2'),
 ('process02', 'resource2'),
 ('process03', 'resource2')]

##### Check for inconsistency

In [54]:
for i in my_list_5:
  tutuple = (i[1],i[0])
  if tutuple in my_list_5:
    print("Inconsistency Found in pair  ", i)
  else:
    print(i, "follows the rule")


('process0', 'resource0') follows the rule
('process03', 'resource0') follows the rule
('process04', 'resource0') follows the rule
('process0', 'resource1') follows the rule
('process01', 'resource1') follows the rule
('process04', 'resource1') follows the rule
('process0', 'resource2') follows the rule
('process02', 'resource2') follows the rule
('process03', 'resource2') follows the rule
